## Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Exploring data

In [5]:
df = pd.read_csv('../../data/processed/data.csv')
df.head()

,name,genre,tomatometer_score,tomatometer_count,audience_score,audience_count,classification,runtime,release_year,original_language
0,A Castle for Christmas,Holiday,0.74,23.0,0.40,100.0,Not Rated,98.0,2021,English
1,Pinocchio,Kids & family,1.00,61.0,0.73,250000.0,G,87.0,1940,English
2,The Informer,Mystery & thriller,0.64,58.0,0.60,250.0,R,113.0,2019,English
3,They Cloned Tyrone,Sci-fi,0.95,129.0,1.00,25.0,R,122.0,2023,English
4,1917,War,0.89,472.0,0.88,25000.0,R,119.0,2019,English


**Question**: For each genre, what would the correlation between tomatometer score and audience be ? 

**Purpose**: To see if there are any genres that are fit for both critics and audience.

In [10]:
genres = np.unique(df['genre'])
genres

array(['Action', 'Adventure', 'Anime', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Game show', 'History',
       'Holiday', 'Horror', 'Kids & family', 'Music', 'Musical',
       'Mystery & thriller', 'Romance', 'Sci-fi', 'War', 'Western'],
      dtype=object)